<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Principles of Time Series Analysis

_Author: Matt Brems, Matthew Garton_

---

### Learning Objectives 
_By the end of the lesson, students should be able to:_
- Define time series and cross-sectional data.
- Describe forecasting, some of its challenges, and when it's advised.
- Generate and interpret time plots.
- Define and identify trend, seasonality, and cyclic patterns in time series data.
- Calculate, plot and interpret autocorrelation and partial autocorrelation.
- Properly fit, generate predictions from, and evaluate a linear time series model.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

# This will allow us to avoid a FutureWarning when plotting.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

## Introduction

In class, much of the data we have worked with is **cross-sectional** data. That is, our data is gathered at one moment in time.

<details><summary>What are some examples of cross-sectional data we've worked with?</summary>

- Ames housing data.
- Titanic data.
- Iris data.
</details>

However, some of the data we've worked with is **time series** data.
- Time series data is data gathered over time.
- You may also hear the terms longitudinal data or serial data.

<details><summary>What are some examples of time series data we've worked with?</summary>
    
- Stock data.
- Forest fires data.
- Reddit data.
</details>

## Time Series Data
Data can be gathered at regular or irregular intervals. (Regular is better!)

However, data that is gathered at _seemingly_ regular intervals can still give us some challenges.
- Monthly data: if we're measuring sales over time, the length of each month can impact (Febuary is 10% shorter than the longest months!)

<img src="./images/airlines.png" alt="drawing" width="500"/>

<img src="./images/airlines_february.png" alt="drawing" width="500"/>

- Missing data can present its own challenges.

<img src="./images/missing_data.png" alt="drawing" width="500"/>

- In cases where our data is irregular, we _may_ be better off attempting to turn irregular data into more regular intervals where possible.
- For example, instead of predicting monthly sales, would it be reasonable to for us to predict quarterly or annual quarterly sales?
    - Quarterly sales range from 90 to 92 days.
    - Annual sales will be 365 or 366 days.

It all depends on what we're trying to **forecast**.

<details><summary>What do you think forecasting is?</summary>

- Forecasting is predicting the future as accurately as possible.
</details>

## Forecasting

Our main focus during time series week is not to conduct inference, but to generate forecasts for future values of one or more variables.

**However, we should first determine whether or not we want to forecast.**
- It's not enough to grab some data and throw it into a model. We have to think more intentionally about what we're trying to do.

- Our model's performance will depend on many things:
    - How much data is available.
    - How well we understand factors that contribute to the thing we want to predict. (a.k.a. how predictable is the thing we want to predict!)
    - Whether forecasts can affect what we want to forecast.

### Notation

In general, we'll use the following notation (which is commonly what you'll see in other resources, too):

$$
\begin{eqnarray*}
Y_t &=& \text{the value of } Y \text{ at time } t \\
Y_{t-1} &=& \text{the value of } Y \text{ at time } t-1 \\
\text{as usual, } &Y& \text{is the thing we want to predict!} \\
\\
X_t &=& \text{the value of } X \text{ at time } t \\
\text{as usual, } &X& \text{is something that may help us predict } Y
\end{eqnarray*}
$$

We'll use models that can help us forecast time series.
- Using past $Y$ values to predict future $Y$ values.
- Using independent $X$ variables to predict future $Y$ values.
- Using both past $Y$ values and independent $X$ variables to predict future $Y$ values.

However, these bring up a lot of questions.
- How many $Y$ values should be used when predicting future $Y$ values?
- What $X$ variables should I use to predict future $Y$ values?
- Depending on how far forward I want to forecast, will my desired $X$ variables even be available?
    - Maybe I think Facebook's stock price today is a good predictor of Google's stock price tomorrow. If I want to predict Google's stock price 50 days from now, will I have access to Facebook's stock price 49 days from now? (No!)
    - The further ahead we forecast, the more uncertain we are.
    
In order to help us answer the above questions, **we need to plot our data**.

In [ ]:
# Instantiate dataframe with index.
df =

# Set seed.
np.random.seed(42)

# Generate data.
data = [1]
for i in range(1,df.shape[0]):
    data.append(data[i-1] + np.random.normal())
    
# Put data in DataFrame.
df['co2'] = data

## Plotting Time Series

Plotting time series is hugely important. By plotting our time series, we can visualize short-term and long-term behavior, which significantly informs which model(s) we should fit.

The most common type of plot is called a **time plot**.

<img src="./images/airlines.png" alt="drawing" width="500"/>

- A time plot is a line plot with time on the horizontal axis and the value of our series on the vertical axis.

To make things easier, we're going to write a function that will generate plots for us.

In [ ]:
# Write function called plot_series that takes in 
# a dataframe, a list of column names to plot, the 
# plot title and the axis labels as arguments,
# then displays the line plot with a figure size
# of 18 horizontal inches by 9 vertical inches.

def plot_series(df, cols=None, title='Title', xlab=None, ylab=None):
    
    # Set figure size to be (18, 9).
    plt.figure(figsize=(18,9))
    
    # Iterate through each column name.
    for col in cols:
        
        # Generate a line plot of the column name.
        # You only have to specify Y, since our
        # index will be a datetime index.
        plt.plot(df[col])
        
    # Generate title and labels.
    plt.title(title, fontsize=26)
    plt.xlabel(xlab, fontsize=20)
    plt.ylabel(ylab, fontsize=20)
    
    # Enlarge tick marks.
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18);

In [ ]:
plot_series()

<details><summary>How would you describe this plot?</summary>
    
_(Answers will vary.)_

- Across the full range of time, the value of $Y$ increases.
- There is a sharp jump upward around July 2019.
- $Y$ increases and decreases over time, but in irregular patterns.
- The values of $Y_t$ and $Y_{t-1}$ are highly correlated with one another.
</details>

## Describing the Behavior of a Time Series

When attempting to forecast time series data, it is critical for us to describe the behavior of the data. (*This is going to tell us how to model our data!*)

We generally plot the data, then identify whether or not a few things are exhibited: trends, seasonality, cyclical patterns, and autocorrelation.

### Trends

In time series data, a **trend** is the long-term increase or decrease in the data.

The two most common types of trends will be **linear (straight line) trends** and **exponential trends**.

<img src="./images/column_1_small.png" alt="drawing" width="550"/>

<details><summary>Is there a trend in the change in carbon dioxide output? If so, do you think it's a straight line trend or an exponential trend?</summary>

- Yes, there is a trend in the graph. There appears to be a long-term increase over time.
- The trend could be either linear or exponential!
</details>

We'll decide how to model the above shortly.

### Seasonality

In time series data, **seasonality** describes when a time series is affected by factors that take on a **fixed and known frequency**.

Seasonality does not have to explicitly refer to seasons of the year (and from an inclusivity standpoint, different regions of the world have different seasons), but seasonality almost always **will** be tied to some meaning on the calendar!

<details><summary>Can you think of examples of time series data that exhibit seasonality?</summary>
    
- Sales data for many products likely exhibits seasonality. Purchases of heavier coats will probably peak in colder times of the year, which will occur roughly every twelve months.
- Restaurant sales likely exhibits seasonality with a peak every week (usually Friday or Saturday).
- Temperature itself would exhibit seasonality! We expect temperature to fluctuate up and down in a relatively consistent pattern every twelve months or so.
    - Note that temperature also has an increasing trend... so trends and seasonality can occur together!
</details>

<img src="./images/column_1_small.png" alt="drawing" width="550"/>

<details><summary>Is there seasonality in the change in carbon dioxide output? If so, about how frequently?</summary>

- No, there does not appear to be seasonality.
- While $Y$ does fluctuate up and down, it does not appear to fluctuate with some fixed and known frequency.
</details>

### Cyclic Patterns

In time series data, a **cyclic pattern** refers to fluctuations that are **not** of a fixed and known frequency.

<img src="./images/column_1_small.png" alt="drawing" width="550"/>

<details><summary>Is there a cyclic pattern in the change in carbon dioxide output?</summary>

- Yes, there is a cyclic pattern. $Y$ does fluctuate up and down and there is no fixed and known frequency.
</details>

One tool that _can_ be helpful is the `seasonal_decompose` tool from `statsmodels`. However, we have to be careful!

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
# Decompose time series into trend, seasonal, and residual components.
decomp =

# Plot the decomposed time series.
;

By default, the decomposition will assume there is a linear trend. If you believe there is an exponential trend, you should change the argument `model` to be `multiplicative`, but this will fail with values that are less than or equal to 0.

The plot will **always** pull out a seasonal component... even if there isn't a seasonal component in our data! In short, this is sometimes a helpful tool, but it's not a substitute for plotting your data with a time plot and examining it first.

### Autocorrelation

When fitting a linear regression model, there are five assumptions when fitting that model. Some of these are violated by time series data!

<details><summary>Which assumption of a multiple linear regression model do you think is most violated by time series data?</summary>

- While any of the assumptions can be violated by time series data, the most common one to be violated is **independence of errors**.
- Suppose I fit a linear regression model to our data and have a particularly large residual (error) at time $t=5$. There's a good chance I also have a large residual at time $t=4$ and time $t=6$.
</details>

In time series data, autocorrelation refers to the **correlation of one variable with lagged versions of itself**. (You may also hear the term **serial correlation**.)

You might imagine a dataset that looks like this:

|  $t$  | $Y_t$ | $Y_{t-1}$ | $Y_{t-2}$ | $Y_{t-3}$ |
|-------|-------|-----------|-----------|-----------|
| 1     | $y_1$ | NaN       | NaN       | NaN       |
| 2     | $y_2$ | $y_1$     | NaN       | NaN       |
| 3     | $y_3$ | $y_2$     | $y_1$     | NaN       |
| 4     | $y_4$ | $y_3$     | $y_2$     | $y_1$     |

In [ ]:
# Generate a dataframe with our CO2 data that
# looks like the above dataframe.

df['co2_lag_1'] = 
df['co2_lag_2'] = 
df['co2_lag_3'] = 

In [ ]:
df.head()

We can calculate the correlation between $Y_t$ and $Y_{t-1}$, which tells us how highly correlated values of $Y$ are with the immediately previous value of $Y$.

<details><summary>BONUS: If you want to see the formula for autocorrelation, check here.</summary>
    
$$
\begin{eqnarray*}
Corr(Y_t, Y_{t-k}) = \frac{Cov(Y_t,Y_{t-k})}{Var(Y_t)Var(Y_{t-k})}
\end{eqnarray*}
$$
</details> 

In [ ]:
# Calculate the correlations among these columns.


We can also calculate the correlation between $Y_t$ and $Y_{t-2}$, $Y_t$ and $Y_{t-3}$, and so on. But this can be tedious to do by hand, so we generally use a function to generate an **autocorrelation plot** that visualizes this all at once!

In [ ]:
# Import the autocorrelation function (ACF) plot.
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
# Generate the ACF plot on co2 data
# up to 20 time periods.
;

- The left-most value of the autocorrelation plot is $Corr(Y_t, Y_t)$. This should always be 1! (The height of the blue dot will always be 1.)
- The next value (moving one step to the right) of the autocorrelation plot visualizes $Corr(Y_t, Y_{t-1})$. The height of the blue dot here is about 0.99.
- The next value of the autocorrelation plot visualizes $Corr(Y_t, Y_{t-2})$. The height of the blue dot here is about 0.98.
- The correlation between $Y_t$ and $Y_{t-20}$ is still above 0.9, which is quite high!
- The blue band represents a 95% confidence interval. If the value of the autocorrelation goes outside the blue band, that means that correlation is statistically significant. _(With a small caveat, which we'll discuss shortly.)_

#### Interpretation

The autocorrelation plot is an important tool for a few reasons:
- **We can use it to detect trends**: When the autocorrelation plot has large and positive values for small lags (e.g. lags 1, 2, 3), this is evidence that a trend exists.
- **We can use it to detect seasonality**: Recall that seasonality means we have fluctuations in our $Y$ variable that occur with a fixed and known frequency. When the autocorrelation plot has larger values for the seasonal lags than other lags, this is evidence that seasonality exists. (Visually, this might look like a scalloped shape.)
- **We can use this plot to inform our modeling choices**, along with a related plot - the partial autocorrelation function plot.

### Partial Autocorrelation

Based on the autocorrelation plot above, we might try to fit a model that looks like this:

$$
\begin{eqnarray*}
Y_t &=& \beta_0 + \beta_1Y_{t-1} + \beta_2Y_{t-2} + \cdots + \beta_{20}Y_{t-20} \\
&=& \beta_0 + \sum_{k=1}^{20}\beta_kY_{t-k}
\end{eqnarray*}
$$



<details><summary>However, this can run into many problems!</summary>

- The more variables we have, the likelier we are to overfit our model to the data.
- $Y_{t-1}$ and $Y_{t-2}$ and $Y_{t-3}$ are all highly correlated with one another, so the inputs to our model in this case aren't independent of one another.
- When we use a variable like $Y_{t-20}$, we effectively decrease our sample size by 20. (If our data are measured year over year, discarding 20 years of data is **a lot of data** to exclude from our model.)
</details>

To avoid this problem, we use the **partial autocorrelation function**.
- The partial autocorrelation is like the autocorrelation in that it checks for the correlation between $Y_t$ and lagged versions of itself.
- However, the partial autocorrelation **controls for all lower-lag autocorrelations**.
    - That is, the partial autocorrelation between $Y_t$ and $Y_{t-2}$ is the correlation between $Y_t$ and $Y_{t-2}$  that has already taken into account the autocorrelation between $Y_t$ and $Y_{t-1}$.
    
    
<details><summary>BONUS: If you want to see the formula for partial autocorrelation, check here.</summary>
    
$$
\begin{eqnarray*}
PartialCorr(Y_t, Y_{t-k}) = \frac{Cov(Y_t,Y_{t-k}|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})}{Var(Y_t|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})Var(Y_{t-k}|Y_{t-1},Y_{t-2},\ldots,Y_{t-(k-1)})}
\end{eqnarray*}
$$
</details>     

In [ ]:
# Import the partial autocorrelation function (PACF) plot.
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
# Generate the PACF plot on CO2 data
# up to 20 time periods.
;

- The blue band represents a 95% confidence interval. If the value of the partial autocorrelation goes outside the blue band, that means that correlation is statistically significant.

#### Interpretation

The partial autocorrelation plot is an important tool for a few reasons:
- **We can use it to detect seasonality**: Recall that seasonality means we have fluctuations in our $Y$ variable that occur with a fixed and known frequency. When the partial autocorrelation plot has significant values for the seasonal lags, this is evidence that seasonality exists.
- **We can use this plot to inform our modeling choices**: Here, we see that by using $Y_{t-1}$ as a predictor for $Y_t$, we probably won't get much important information if we were to also include $Y_{t-2}$ (or higher order lags) as a predictor.

## Recap

When you look at a new time series, you should:
- generate a time plot.
- generate the autocorrelation plot.
- generate the partial autocorrelation plot.

Use these plots to determine:
- is there a trend?
- is there seasonality?
- is there evidence of cyclic patterns?

Once we answer these questions, we can begin to model.

## Practice

On your own, take 7 minutes to read in the following data and answer the following questions.

In [ ]:
# Read in airline data.
airline_passengers = pd.read_csv('./data/airline-passengers.csv',
                                 parse_dates=['Month'])

# Set the index to be month over month.


In [ ]:
airline_passengers.head()

In [ ]:
# Generate a time plot of the airline passengers data.


In [ ]:
# Generate a decomposition plot of the airline passengers data.


In [ ]:
# Generate an ACF plot of the airline passengers data with 30 time periods.


In [ ]:
# Generate an PACF plot of the airline passengers data with 30 time periods.


<details><summary>1. Based on the time plot, is there evidence of a trend? If so, describe the trend. How can you tell?</summary>
    
- There is a trend. It seems to be increasing and pretty linear.
</details>

<details><summary>2. Based on the time plot, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?</summary>
    
- There is evidence of seasonality. There appear to be peaks in the data about every twelve months. This makes sense, because the number of people flying on planes in a given month may peak when there's more travel (e.g. summer) and it might die down in other months.
- This evidence is strongest in the later years, where from 1954-1961 there's obviously one peak every year. (Note that there are two peaks between 1956 and 1958, two peaks between 1958 and 1960, etc.)
</details>

<details><summary>3. Based on the time plot, is there evidence of a cyclic pattern? How can you tell?</summary>
    
- This one is harder to tell! Maybe yes, maybe no.
- To more definitively answer this, I'm going to use the `seasonal_decomposition` to see if, after removing the trend and seasonal pieces, there appears to be some cyclic pattern in the residuals.
- Based on the decomposition below, it does appear that there's some cyclical pattern. The trend and seasonality by themselves explain a lot fo what happens between 1954 and 1956, but as we move farther away from 1955, our trend/seasonality appear to explain less and less of what's going on in our data.
</details>

<details><summary>4. Based on the ACF plot, is there evidence of a trend? If so, describe the trend. How can you tell?</summary>

- Yes, there is a trend. In the ACF plot, the small lag values have large, positive autocorrelations.
</details>

<details><summary>5. Based on the ACF and PACF plots, is there evidence of seasonality? If so, estimate the frequency of the seasonality. How can you tell?</summary>
    
- Yes, there is seasonality. In the ACF plot, there is a "scalloped" shape, where we see ACF values peak roughly every 12 months.
- In the PACF plot, we see significant values every 12 months or so. We also see some positive and some negative significant partial autocorrelations, which usually indicates strong seasonal fluctuations.
</details>

<details><summary>Based on the above plots, we might conclude:</summary>

- The last two values are highly informative in predicting future values.
- There is a linear trend in our data.
- There is seasonality that occurs roughly every twelve months.

Let's fit the following model:
$$
\begin{eqnarray*}
Y_t &=& \beta_0 + \beta_1Y_{t-1} + \beta_2Y_{t-2} + \beta_3Y_{t-12} + \beta_4t
\end{eqnarray*}
$$
</details>

In [ ]:
airline_passengers.head()

In [ ]:
# Create a column called `lag_1` that lags Passengers by one month.
airline_passengers['lag_1'] = 

# Create a column called `lag_2` that lags Passengers by two months.
airline_passengers['lag_2'] = 

# Create a column called `seasonal_12` that lags Passengers by one year.
airline_passengers['seasonal_12'] = 

# Create a variable called `time` that takes on a value of 0 in January 1949,
# then increases by 1 each month until the end of the dataframe.
airline_passengers['time'] = 

In [ ]:
# Confirm the top of our dataframe looks good.
airline_passengers.head()

In [ ]:
# Confirm the bottom of our dataframe looks good.
airline_passengers.tail()

## Split Data into X and y


In [ ]:
X = airline_passengers.drop(columns='Passengers')
y = airline_passengers['Passengers']

Note: We won't always do this! If we're only using our $Y$ variable to predict our $Y$ variable, we may not need to manually engineer features into an $X$ dataframe, but the model may handle that. We'll get into these models later!

## Train-Test Split

Before building a model, we should split our data up into a training and testing set.

Since our goal with time series models is almost always to forecast values forward in time, the idea with a time series train/test split is to train on earlier data and test/evaluate on later data.

Most commonly, we'll set our:
- training set to be the "first" 67% - 80% of our data timewise.
- test set be the "last" 20% - 33% timewise.

Let's split our dataframe by taking the first 80% of rows for training and the rest for testing.

In [ ]:
# Generate train/test split.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2)

In [ ]:
# Check shape to confirm we did this properly.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Fit a linear model.

In [ ]:
# Import statsmodels.
import statsmodels.api as sm

In [ ]:
# Before fitting a model in statsmodels, what do we need
# to do? (Hint: Think intercept.)



# Confirm.


In [ ]:
# statsmodels won't be able to handle missing values.



In [ ]:
# Remember that, in statsmodels, we pass our data 
# in when we instantiate the model!

lm =

In [ ]:
# Then we fit our model.
lm_results =

In [ ]:
# Display our summary!
print(lm_results.summary())

In [ ]:
# Generate predicted test values.


In [ ]:
# Import R2 score and MSE.
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# Calculate R2 score.
r2_score()

In [ ]:
# Calculate RMSE.
mean_squared_error()

In [ ]:
# Let's plot our predictions! 

# Set figure size.
plt.figure(figsize=(20,10))

# Plot training data.
plt.plot(y_train.index, y_train.values, color = 'blue')

# Plot testing data.
plt.plot(y_test.index, y_test.values, color = 'orange')

# Plot predicted test values.
plt.plot(lm_results.predict(X_test), color = 'green')

# Set label.
plt.title(label = 'Forecasting Airline Passengers 1958-1961', fontsize=24)

# Resize tick marks.
plt.xticks(fontsize=20)
plt.yticks(fontsize=20);

### Learning Objectives 
_By the end of the lesson, students should be able to:_
- Define time series and cross-sectional data.
- Describe forecasting, some of its challenges, and when it's advised.
- Generate and interpret time plots.
- Define and identify trend, seasonality, and cyclic patterns in time series data.
- Calculate, plot and interpret autocorrelation and partial autocorrelation.
- Properly fit, generate predictions from, and evaluate a linear time series model.